# MODEL TRAINING  USING THE TF OBJECT DETECTION API

This notebook is based on modular [shell scripts](./scripts) for convenience. The scripts comprise minimal lines of code (LOC), on average, no more than 10 LOC for each, making them easy to read, learn, and adapt for future applications.

**!!!Caveat :** 

Some scripts can be conveniently executed directly from this notebook, and some need to be executed in terminal to grant permissions. I recommend running each script from the terminal to see STDOUT clearly. If run in terminal, use the [module-training](./) directory as the working directory.

***This is an academic project. Interested developers are invited to freely contribute issues, questions, improvements, and discussions.***

## 1. CHECK SUBMODULES

[`check-submodules.sh`](./scripts/) makes sure that the [LabelImg](https://github.com/tzutalin/labelImg) and [TF models](https://github.com/tensorflow/models) submodule repositories are initialized correctly.

In [2]:
!./scripts/check-submodules.sh

## 2. CREATE VIRTUAL ENVIRONMENT

[`venv.sh`](./scripts/) creates a virtual environment and installs essential [apt](./resources/apt.txt) and [pip](./resources/pip.txt) dependencies into the environment. As such, the script should be executed in terminal to grant install permissions [( see dependencies.sh )](./scripts/). By default, the name of the virtual environment is **tfod-venv**. This name can, of cause, be changed. From working directory run

```
./scripts/venv.sh
```

## 3. INSTALL TF OBJECT DETECTION

This command can also be run in terminal to leverage clear STDOUT, which is critical to verify correct installation.

In [ ]:
!./scripts/tfod.sh

## 4. GET PRETRAINED MODEL FROM [TENSOR FLOW MODEL ZOO](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

[`pretrained-model.sh`](./scripts/) uses *wget* to download a pre-trained model from the [zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). The script can be adapted to download any pre-trained model.

In [ ]:
!./scripts/pretrained-model.sh

## 5. TRAIN MODEL USING TRANSFER LEARNING 

**!!!CAVEAT :** 

Before proceeding, verify [tallying versions](https://www.tensorflow.org/install/source#gpu) of tensorflow, tensorflow-gpu, cuDNN, and CUDA have been installed. Also remember to link CUDA. E.g., from terminal 

######  # find shared object
```
sudo find / -name 'libcudart.so*'

# example output:
/usr/lib/x86_64-linux-gnu/libcudart.so.10.1
/usr/lib/x86_64-linux-gnu/libcudart.so
```
###### # add it to path 
```
export PATH=/usr/lib/x86_64-linux-gnu${PATH:+:${PATH}}
export LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu:${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
```
###### # set permission
```
sudo chmod a+r /usr/lib/x86_64-linux-gnu/libcuda*
```

### 5.1  Create class labels

In [2]:
import os

# annotations file 
ann_file = os.path.join('annotations/annotations.pbtxt')

# create class labels
labels = [{'name':'smartphone', 'id':1}]

# write class labels to annotations file
with open(ann_file, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### 5.2 Create TF records

In [3]:
!./scripts/tfrecords.sh

Successfully created the TFRecord file: /home/everett/Repositories/research/projects/test/TFOD-model-training/model-training/records/train.record
Successfully created the TFRecord file: /home/everett/Repositories/research/projects/test/TFOD-model-training/model-training/records/test.record


### 5.3 Copy pipeline configuration of pretrainded model

In [6]:
import os
pretrained_pipeline_config_file = os.path.join('models/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config')
models = os.path.join('models')

In [7]:
!cp {pretrained_pipeline_config_file} {models}

### 5.4 Configure pipeline

In [5]:
# import TS object detection TODO: TESTING FROM HERE ONWARDS!!!
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

model_pipeline_config_file = os.path.join('models/pipeline.config')
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

# parse cloned pipeline config
with tf.io.gfile.GFile(model_pipeline_config_file, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 
    
# update pipeline settings
model_test_record = [os.path.join('records/train.record')]
model_train_record = [os.path.join('records/train.record')]
model_checkpoint = os.path.join('models/pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0')

pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4

pipeline_config.train_input_reader.label_map_path= ann_file
pipeline_config.train_config.fine_tune_checkpoint = model_checkpoint
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = model_train_record

pipeline_config.eval_input_reader[0].label_map_path = ann_file
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = model_test_record

config_text = text_format.MessageToString(pipeline_config)  

with tf.io.gfile.GFile(model_pipeline_config_file, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 

### 5.5 (Optional) Verify new configuration (untested)

In terminal run

```
./scripts/pipelineconfig.sh
```

### 5.6 Train model

[`train.sh`](./scripts/) is better run in terminal to see STDOUT clearly.

In [ ]:
!./scripts/train.sh

### 5.7 (Optional) Evaluate model

In terminal run

```
./scripts/evaluate.sh
```

### 5.8. Load pipeline config and restore chekpoint

In [76]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [78]:
# load pipeline config and build model
configs = config_util.get_configs_from_pipeline_file(model_pipeline_config_file)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
model_checkpoint = os.path.join('models/ckpt-3') ## todo: dynamically find last checkpoint
ckpt.restore(model_checkpoint).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections